In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


In [2]:
email = pd.read_csv('Suicide_Detection.csv')
email.head()

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [3]:
email.shape

(232074, 3)

In [4]:
# Drop the 'Unnamed: 0' column
email.drop(columns=['Unnamed: 0'], inplace=True)

In [5]:
email.isnull().sum()

text     0
class    0
dtype: int64

In [6]:
from wordcloud import WordCloud, STOPWORDS

In [9]:
#check dubplicat
email.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
232069    False
232070    False
232071    False
232072    False
232073    False
Length: 232074, dtype: bool

In [10]:
#check 
print(email[email.duplicated()])

Empty DataFrame
Columns: [text, class]
Index: []


In [11]:
email.duplicated().sum()

0

In [12]:
email.drop_duplicates(inplace=True)

In [14]:
# replace the null values with a null string
email = email.fillna('')

In [15]:
#encoding
# Map 'ham' to 0 and 'spam' to 1
email['class'] = email['class'].map({'suicide': 0, 'non-suicide': 1})

# Split the data into features (X) and labels (y)
X = email['text']
y = email['class']

# Convert features and labels to numpy arrays
X = np.array(X)
y = np.array(y)
X

array(["Ex Wife Threatening SuicideRecently I left my wife for good because she has cheated on me twice and lied to me so much that I have decided to refuse to go back to her. As of a few days ago, she began threatening suicide. I have tirelessly spent these paat few days talking her out of it and she keeps hesitating because she wants to believe I'll come back. I know a lot of people will threaten this in order to get their way, but what happens if she really does? What do I do and how am I supposed to handle her death on my hands? I still love my wife but I cannot deal with getting cheated on again and constantly feeling insecure. I'm worried today may be the day she does it and I hope so much it doesn't happen.",
       "Am I weird I don't get affected by compliments if it's coming from someone I know irl but I feel really good when internet strangers do it",
       'Finally 2020 is almost over... So I can never hear "2020 has been a bad year" ever again. I swear to fucking God it\'

In [16]:
y = y.reshape(-1,1)
y

array([[0],
       [1],
       [1],
       ...,
       [1],
       [0],
       [1]], dtype=int64)

In [17]:
print(tabulate(email.head(), headers=email.columns, tablefmt="fancy_grid"))

╒════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

## Buil Model

In [18]:
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Define the sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Define the cost function
def cost(h, y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

# Define the gradient descent function
def gradient_descent(X, h, y):
    return np.dot(X.T, (h - y)) / y.shape[0]

# Define the update function
def update_weights(weight, learning_rate, gradient):
    return weight - learning_rate * gradient

# Define the prediction function
def predict(X, weight):
    return sigmoid(np.dot(X, weight))

# Define the training function
def train(X, y, weight, learning_rate, iterations):
    for i in range(iterations):
        h = predict(X, weight)
        gradient = gradient_descent(X, h, y)
        weight = update_weights(weight, learning_rate, gradient)
        
        if(i % 10 == 0):
            print(f'cost: {cost(h, y)} \t')
    return weight

In [20]:
import nltk
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
email['text'] = email['text'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
from sklearn.decomposition import TruncatedSVD

vectorizer = TfidfVectorizer()
X_sparse = vectorizer.fit_transform(email['text'])

# Reduce to 100 components (you can adjust this number)
svd = TruncatedSVD(n_components=100)
X_reduced = svd.fit_transform(X_sparse)


In [24]:
X

array(["Ex Wife Threatening SuicideRecently I left my wife for good because she has cheated on me twice and lied to me so much that I have decided to refuse to go back to her. As of a few days ago, she began threatening suicide. I have tirelessly spent these paat few days talking her out of it and she keeps hesitating because she wants to believe I'll come back. I know a lot of people will threaten this in order to get their way, but what happens if she really does? What do I do and how am I supposed to handle her death on my hands? I still love my wife but I cannot deal with getting cheated on again and constantly feeling insecure. I'm worried today may be the day she does it and I hope so much it doesn't happen.",
       "Am I weird I don't get affected by compliments if it's coming from someone I know irl but I feel really good when internet strangers do it",
       'Finally 2020 is almost over... So I can never hear "2020 has been a bad year" ever again. I swear to fucking God it\'

In [26]:
y = np.array(email['class'])
y


array([0, 1, 1, ..., 1, 0, 1], dtype=int64)

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X = email['text']  # Replace 'text_column' with the actual name
y = email['class']  # Replace 'label_column' with the actual name

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [33]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_tfidf, y_train)


LogisticRegression()

In [34]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.94      0.93      0.94     23128
           1       0.93      0.95      0.94     23287

    accuracy                           0.94     46415
   macro avg       0.94      0.94      0.94     46415
weighted avg       0.94      0.94      0.94     46415



In [35]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
